In [1]:
using Distributed
addprocs(4)

@everywhere begin
    using Revise
    using MendelIHT
    using SnpArrays
    using Random
    using GLM
    using DelimitedFiles
    using Test
    using Distributions
    using LinearAlgebra
    using CSV
    using DataFrames
    using StatsBase
    BLAS.set_num_threads(1) # remember to set BLAS threads to 1 !!!
#     using TraitSimulation, OrdinalMultinomialModels, VarianceComponentModels
end

# Simple multivariate Gaussian traits

With $r$ traits, each sample's phenotype $\mathbf{y}_{i} \in \mathbb{R}^{n \times 1}$ is simulated under

$$\mathbf{y}_{i}^{r \times 1} \sim N(\mathbf{B}^{r \times p}\mathbf{x}_{i}^{p \times 1}, \ \ \Sigma_{r \times r})$$

This model assumes each sample is independent.

In [2]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs
r = 2     # number of traits

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray("multivariate_$(r)traits.bed", n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n, 1)
intercepts = [10.0 1.0] # each trait have different intercept

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r, Zu=z*intercepts, overlap=2);

In [3]:
# save true SNP's position and effect size
open("multivariate_$(r)traits_true_beta.txt", "w") do io
    println(io, "snpID,effectsize")
    for pos in correct_position
        println(io, "snp$pos,", true_b[pos])
    end
end

# create `.bim` and `.bam` files using phenotype
make_bim_fam_files(x, Y, "multivariate_$(r)traits")

# save phenotypes in separate file
open("multivariate_$(r)traits.phen", "w") do io
    println(io, "FID\tIID\tT1\tT2")
    for i in 1:n
        println(io, "$i\t1\t", Y[i, 1], "\t", Y[i, 2])
    end
end

## Run IHT

In [4]:
Yt = Matrix(Y')
Zt = Matrix(z')
ktrue = k + count(!iszero, intercepts)
@time result = fit_iht(Yt, Transpose(xla), Zt, k=12, verbose=true)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 12
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 200
Converging when tol < 0.0001:

Iteration 1: loglikelihood = 215.4892687838203, backtracks = 0, tol = 0.1243455904380314
Iteration 2: loglikelihood = 1382.3724288548506, backtracks = 0, tol = 0.027019894208788985
Iteration 3: loglikelihood = 1477.7383135165255, backtracks = 0, tol = 0.014225517157910438
Iteration 4: loglikelihood = 1511.714843337415, backtracks = 0, tol = 0.0044564


Compute time (sec):     0.14228582382202148
Final loglikelihood:    1521.6067421001371
Iterations:             15
Trait 1's SNP PVE:      0.554527358091918
Trait 2's SNP PVE:      0.61958796264493

Trait 1: IHT estimated 4 nonzero SNP predictors
4×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     1197     0.121446
   2 │     5651    -0.200705
   3 │     5797    -1.09767
   4 │     8087     1.2791

Trait 1: IHT estimated 1 non-genetic predictors
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1       10.027

Trait 2: IHT estimated 6 nonzero SNP predictors
6×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      326     0.331882
   2 │     2110     0.575645
   3 │     5375     1.19357
   4 │     5797     0.502072
   5 │     6015     0.81844
   6 │     6813     1.36932

Trait 2: IHT estimated 1 non-

## Check answer

In [5]:
# first beta
β1 = result.beta[1, :]
true_b1_idx = findall(!iszero, true_b[:, 1])
[β1[true_b1_idx] true_b[true_b1_idx, 1]]

4×2 Array{Float64,2}:
 -0.200705  -0.224675
 -1.09767   -1.14044
  0.0       -0.14698
  1.2791     1.25668

In [6]:
# second beta
β2 = result.beta[2, :]
true_b2_idx = findall(!iszero, true_b[:, 2])
[β2[true_b2_idx] true_b[true_b2_idx, 2]]

6×2 Array{Float64,2}:
 0.331882  0.315219
 0.575645  0.609812
 1.19357   1.20121
 0.502072  0.531549
 0.81844   0.808327
 1.36932   1.43455

In [7]:
# non genetic covariates
[result.c intercepts']

2×2 Array{Float64,2}:
 10.027    10.0
  1.03625   1.0

In [8]:
# covariance matrix
[vec(result.Σ) vec(true_Σ)]

4×2 Array{Float64,2}:
  2.4681    2.53934
 -1.83681  -1.85399
 -1.83681  -1.85399
  2.42177   2.41416

In [9]:
# number of causal SNPs recovered
correct_snps = [x[1] for x in correct_position]             # truely causal snps
signif_snps = findall(!iszero, β1) ∪ findall(!iszero, β2)   # IHT's selected snps
signif_snps ∩ correct_snps

8-element Array{Int64,1}:
 5651
 5797
 8087
  326
 2110
 5375
 6015
 6813

## Test Cross validation

In [11]:
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time mses = cv_iht(Yt, Transpose(xla), Zt);

Cross validating...100%|████████████████████████████████| Time: 0:00:07




Crossvalidation Results:
	k	MSE
	1	2888.7160633632484
	2	2560.1358620535425
	3	2067.943067029389
	4	1812.0395079444286
	5	1554.312036744936
	6	1277.3237598020085
	7	1154.9320629872832
	8	1098.5910963871872
	9	1019.4597637296985
	10	1030.14124647156
	11	1023.5545874904793
	12	1007.9022110997687
	13	1012.6193656356761
	14	1019.1491606608182
	15	1024.6877890077092
	16	1022.9300595671256
	17	1040.0286509856787
	18	1033.345570850089
	19	1039.8828186471897
	20	1036.274158344765

Best k = 12

  7.249031 seconds (35.04 k allocations: 7.841 MiB)


**Conclusion:** Using $k = 12$ (which achieves minimum MSE) IHT finds 8/10 causal SNPs + 2 intercept terms. Beta estimates and cross-trait covariances are very precise. 

# GEMMA multivariate results

In [84]:
gemma_df = CSV.read("gemma.result.assoc.txt", DataFrame)

# pvalues
pval_wald = gemma_df[!, :p_wald]
pval_lrt = gemma_df[!, :p_lrt]
pval_score = gemma_df[!, :p_score]

# estimated beta
estim_β1 = gemma_df[!, :beta_1]
estim_β2 = gemma_df[!, :beta_2]

# estimated covariance matrix
estim_σ11 = gemma_df[!, :Vbeta_1_1]
estim_σ12 = gemma_df[!, :Vbeta_1_2]
estim_σ22 = gemma_df[!, :Vbeta_2_2];

correct_snps = [x[1] for x in correct_position]  # truely causal snps
signif_snps = findall(x -> x ≤ 0.05/p, pval_lrt) # gemma's selected snps
signif_snps ∩ correct_snps

6-element Array{Int64,1}:
 2110
 5375
 5797
 6015
 6813
 8087

**Conclusion:** GEMMA finds 6/10 causal SNPs

# MV-PLINK

In [25]:
mvplink_df = CSV.read("plink.mqfam.total", DataFrame, delim=' ', ignorerepeated=true)

# pvalues
pval = mvplink_df[!, :P]

# SNPs passing threshold
signif_snps = findall(x -> x ≤ 0.05 / p, pval)
signif_snps ∩ correct_snps

6-element Array{Int64,1}:
 2110
 5375
 5797
 6015
 6813
 8087

**Conclusion:** MV-PLINK finds 6/10 causal SNPs

# More complicated simulations (multivariate traits)

Let us simulate:
+ Non-independent samples
+ Polygenic traits where every SNP contributes to the phenotype, but $k$ SNPs have large effect

For $r$ traits, our model is:

$$\mathbf{Y}_{r \times n} \sim \text{MatrixNormal}(\mathbf{B}_{r \times p}\mathbf{X}_{p \times n}, \ \ \Sigma_{r \times r} , \ \ \sigma_g^2\Phi_{n \times n} + \sigma_e^2 \mathbf{I}_{n \times n})$$

where
+ $\mathbf{X}_{p \times n}$ contains *all* predictors (genetic + non-genetic)
+ $\mathbf{B}_{r \times p}$ contains (true) regression coefficients. Each row is $k$-sparse.
+ $\Sigma_{r \times r}$ is the row (trait) covariance matrix
+ $\sigma_g^2\Phi_{n \times n} + \sigma_e^2 \mathbf{I}_{n \times n}$ is the column (sample) covariance matrix
+ $\Phi$ is the GRM estimated from the genotypes
+ $\sigma_g^2 = 0.6$ and $\sigma_e^2 = 0.4$ (thus heritability is 60%)

In [12]:
"""
k = Number of causal SNPs
p = Total number of SNPs
traits = Number of traits (phenotypes)
overlap = number of causal SNPs shared in each trait
"""
function simulate_random_beta(k::Int, p::Int, traits::Int; overlap::Int=0)
    true_b = zeros(p, traits)
    if overlap == 0
        causal_snps = sample(1:(traits * p), k, replace=false)
        true_b[causal_snps] = randn(k)
    else
        shared_snps = sample(1:p, overlap, replace=false)
        weight_vector = aweights(1 / (traits * (p - overlap)) * ones(traits * p))
        for i in 1:traits
            weight_vector[i*shared_snps] .= 0.0 # avoid sampling from shared snps
        end
        @assert sum(weight_vector) ≈ 1.0
        # simulate β for shared predictors
        for i in 1:traits
            true_b[shared_snps, i] = randn(overlap)
        end
        # simulate β for none shared predictors
        nonshared_snps = sample(1:(traits * p), weight_vector, k - traits * overlap, replace=false)
        true_b[nonshared_snps] = randn(k - traits * overlap)
    end

    return true_b
end

simulate_random_beta

In [13]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs
r = 2     # number of traits

# set random seed for reproducibility
Random.seed!(111)

# simulate `.bed` file with no missing data
x = simulate_random_snparray("multivariate_$(r)traits.bed", n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n, 1)
intercepts = [0.5 1.0] # each trait have different intercept

# simulate β
B = simulate_random_beta(k, p, r, overlap=2)

# between trait covariance matrix
Σ = random_covariance_matrix(r)

# between sample covariance is identity + GRM (2 times because in SnpArrays grm is halved)
Φ = 2grm(x)
σg = 0.6
σe = 0.4
V = σg * Φ + σe * I

# simulate y using TraitSimulations.jl
# VCM_model = VCMTrait(z, intercepts, x, B, [Σ], [V]) #https://github.com/OpenMendel/TraitSimulation.jl/blob/6d1f09c7332471a74b4dd6c8ef2d2b95a96c585c/src/modelframework.jl#L159
# Y = simulate(VCM_model)

# simulate using naive model
μ = z * intercepts + xla * B
Yt = rand(MatrixNormal(μ', Σ, V))

2×1000 Array{Float64,2}:
 6.99773  6.69385  -1.65585  -3.52854   …  -3.11521   -4.13734  -2.32211
 4.49129  5.74267   4.72216   0.575205     -0.585688  -4.44368   4.02092

In [42]:
# create `.bim` and `.bam` files using phenotype
make_bim_fam_files(x, Transpose(Yt), "multivariate_$(r)traits")

# save phenotypes in separate file
open("multivariate_$(r)traits.phen", "w") do io
    println(io, "FID\tIID\tT1\tT2")
    for i in 1:n
        println(io, "$i\t1\t", Yt[1, i], "\t", Yt[2, i])
    end
end

## Run IHT 

In [14]:
Zt = Matrix(z')
ktrue = k + count(!iszero, intercepts)
@time result = fit_iht(Yt, Transpose(xla), Zt, k=17, verbose=true)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 17
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 200
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -208.13036776529475, backtracks = 0, tol = 1.2019658321322253
Iteration 2: loglikelihood = 1730.568644363383, backtracks = 0, tol = 0.090283865811827
Iteration 3: loglikelihood = 2141.9436184153888, backtracks = 0, tol = 0.054978288526687256
Iteration 4: loglikelihood = 2458.6990989456053, backtracks = 0, tol = 0.00736575


Compute time (sec):     0.3098158836364746
Final loglikelihood:    3356.9640313826944
Iterations:             33
Trait 1's SNP PVE:      0.40782795213136286
Trait 2's SNP PVE:      0.8925271478138327

Trait 1: IHT estimated 7 nonzero SNP predictors
7×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     2351   -0.0173457
   2 │     2529   -0.254579
   3 │     2850   -0.0151446
   4 │     2872    0.820841
   5 │     2986   -0.0125593
   6 │     4248   -1.403
   7 │     8921    1.24517

Trait 1: IHT estimated 1 non-genetic predictors
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1     0.506032

Trait 2: IHT estimated 8 nonzero SNP predictors
8×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      359   -0.0123562
   2 │     4710   -0.114557
   3 │     6991   -2.70478
   4 │     8738    0.0126899
   

In [15]:
# first beta
β1 = result.beta[1, :]
true_b1_idx = findall(!iszero, B[:, 1])
[β1[true_b1_idx] B[true_b1_idx, 1]]

5×2 Array{Float64,2}:
 -0.254579  -0.413412
  0.820841   0.811243
  0.0        0.0244199
 -1.403     -1.42827
  1.24517    1.24969

In [16]:
# second beta
β2 = result.beta[2, :]
true_b2_idx = findall(!iszero, B[:, 2])
[β2[true_b2_idx] B[true_b2_idx, 2]]

5×2 Array{Float64,2}:
  0.0       -0.0597678
 -0.114557  -0.11578
 -2.70478   -2.70844
 -0.218876  -0.212269
  0.312726   0.310274

In [17]:
# non genetic covariates
[result.c intercepts']

2×2 Array{Float64,2}:
 0.506032  0.5
 1.00318   1.0

In [18]:
# covariance matrix
[vec(result.Σ) vec(Σ)]

4×2 Array{Float64,2}:
 5.9949    5.65059
 2.0924    1.97568
 2.0924    1.97568
 0.741367  0.702678

In [19]:
# number of causal SNPs recovered
correct_position = findall(!iszero, B)
correct_snps = [x[1] for x in correct_position]             # truely causal snps
signif_snps = findall(!iszero, β1) ∪ findall(!iszero, β2)   # IHT's selected snps
signif_snps ∩ correct_snps

7-element Array{Int64,1}:
 2529
 2872
 4248
 8921
 4710
 6991
 8964

In [21]:
# 4 cores
Random.seed!(2020)
@time mses = cv_iht(Yt, Transpose(xla), Zt, path=1:20, parallel=true, max_iter=20);

Cross validating...100%|████████████████████████████████| Time: 0:00:07




Crossvalidation Results:
	k	MSE
	1	8079.418619716553
	2	3627.3096599448268
	3	1865.7585855184807
	4	1692.7063084360427
	5	1542.7593074646843
	6	1499.8616100579775
	7	1433.6667306296458
	8	1421.9052004472687
	9	1421.2552305804886
	10	1423.4614532506505
	11	1420.2679722482376
	12	1420.0053086973176
	13	1389.0348278471117
	14	1384.3136635494318
	15	1381.4910373174566
	16	1376.2158720886953
	17	1365.8686420435863
	18	1367.0589543865267
	19	1370.884134297744
	20	1370.9739594368748

Best k = 17

  7.276093 seconds (36.59 k allocations: 8.038 MiB)


## GEMMA multivariate results

We use the wald test statistics. The likelihood ratio p-values are all 1.0 or 0.0, so there's some problems with it. 

In [62]:
"""
Computes power and false positive rates
- p: total number of SNPs
- correct_snps: Indices of the true causal SNPs
- detected_snps: Indices of SNPs that are significant after testing
"""
function power_and_fpr(p::Int, correct_snps::Vector{Int}, detected_snps::Vector{Int})
    power = length(signif_snps ∩ correct_snps) / length(correct_snps)
    FP = length(signif_snps) - length(signif_snps ∩ correct_snps) # number of false positives
    TN = p - length(signif_snps) # number of true negatives
    FPR = FP / (FP + TN)
    return power, FP, FPR
end

power_and_fpr

In [83]:
gemma_df = CSV.read("gemma.polygenic.result.assoc.txt", DataFrame)
correct_position = findall(!iszero, B)

# pvalues
pval_wald = gemma_df[!, :p_wald]
pval_lrt = gemma_df[!, :p_lrt]
pval_score = gemma_df[!, :p_score]

# estimated beta
estim_β1 = gemma_df[!, :beta_1]
estim_β2 = gemma_df[!, :beta_2]

# estimated covariance matrix
estim_σ11 = gemma_df[!, :Vbeta_1_1]
estim_σ12 = gemma_df[!, :Vbeta_1_2]
estim_σ22 = gemma_df[!, :Vbeta_2_2];

# check how many real SNPs were recovered
correct_snps = [x[1] for x in correct_position]  # truely causal snps
signif_snps = findall(x -> x ≤ 0.05/p, pval_wald) # gemma's selected snps
@show signif_snps ∩ correct_snps

# compute power, false positives, and false positive rate
power_and_fpr(p, correct_snps, signif_snps)

signif_snps ∩ correct_snps = [2529, 2872, 4248, 6991, 8921]


(0.5, 0, 0.0)

## MV-PLINK

In [82]:
mvplink_df = CSV.read("plink.mqfam.polygenic.total", DataFrame, delim=' ', ignorerepeated=true)

# pvalues
pval = mvplink_df[!, :P]

# SNPs passing threshold
signif_snps = findall(x -> x ≤ 0.05 / p, pval)
@show signif_snps ∩ correct_snps

# compute power, false positives, and false positive rate
power_and_fpr(p, correct_snps, signif_snps)

signif_snps ∩ correct_snps = [2529, 2872, 4248, 6991, 8921]


(0.5, 0, 0.0)

# Conclusion

+ MV-PLINK and IHT run in seconds, GEMMA runs in minutes ~ hours
+ Multivariate IHT's beta estimates are precise, under both truly sparse or polygenic model.
+ With polygenic model, IHT finds more true SNPs but also has higher false positive rate. 
+ In a separate simulation with 5k samples and 300k SNPs, IHT's cross validation takes roughly 1.5h on 8 cores. 

# Large scale simulation

The code below is meant for running on Hoffman2 cluster.

In [1]:
using Distributed
# addprocs(12)

@everywhere begin
    using Revise
    using MendelIHT
    using SnpArrays
    using Random
    using GLM
    using DelimitedFiles
    using Distributions
    using LinearAlgebra
    using CSV
    using DataFrames
    using StatsBase
end

"""
k = Number of causal SNPs
p = Total number of SNPs
traits = Number of traits (phenotypes)
overlap = number of causal SNPs shared in each trait
"""
function simulate_random_beta(k::Int, p::Int, traits::Int; overlap::Int=0)
    true_b = zeros(p, traits)
    if overlap == 0
        causal_snps = sample(1:(traits * p), k, replace=false)
        true_b[causal_snps] = randn(k)
    else
        shared_snps = sample(1:p, overlap, replace=false)
        weight_vector = aweights(1 / (traits * (p - overlap)) * ones(traits * p))
        for i in 1:traits
            weight_vector[i*shared_snps] .= 0.0 # avoid sampling from shared snps
        end
        @assert sum(weight_vector) ≈ 1.0
        # simulate β for shared predictors
        for i in 1:traits
            true_b[shared_snps, i] = randn(overlap)
        end
        # simulate β for none shared predictors
        nonshared_snps = sample(1:(traits * p), weight_vector, k - traits * overlap, replace=false)
        true_b[nonshared_snps] = randn(k - traits * overlap)
    end

    return true_b
end

"""
# Arguments
xla = simulated genotype matrix (converted to a SnpLinAlg)
k = number of causal SNPs
r = number of traits
Φ = estimated GRM (using GEMMA)

# Optional arguments
seed = random seed for reproducibility
σ2 = contribution of GRM
σe = random environmental effect
βoverlap = number of causal SNPs shared in all traits
"""
function simulate_multivariate_polygenic(
    plinkname::String, n::Int, p::Int, k::Int, r::Int;
    seed::Int=2021, σg=0.6, σe=0.4, βoverlap=2, 
    )
    # set seed
    Random.seed!(seed)
    
    # simulate `.bed` file with no missing data
    x = simulate_random_snparray("sim$seed/" * plinkname * ".bed", n, p)
    xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true)
    
    # intercept is the only nongenetic covariate
    Z = ones(n, 1)
    intercepts = randn(r)' # each trait have different intercept

    # simulate β
    B = simulate_random_beta(k, p, r, overlap=βoverlap)
    writedlm("sim$(seed)/trueb.txt", B)

    # between trait covariance matrix
    Σ = random_covariance_matrix(r)

    # between sample covariance is identity + GRM (2x because OpenMendel always uses half the GRM)
    Φ = 2grm(x, method=:Robust)
    writedlm("sim$seed/$plinkname.grm", Φ)
    V = σg * Φ + σe * I

    # simulate y using TraitSimulations.jl
    # VCM_model = VCMTrait(Z, intercepts, x, B, [Σ], [V]) #https://github.com/OpenMendel/TraitSimulation.jl/blob/6d1f09c7332471a74b4dd6c8ef2d2b95a96c585c/src/modelframework.jl#L159
    # Y = simulate(VCM_model)

    # simulate using naive model
    μ = Z * intercepts + xla * B
    Y = rand(MatrixNormal(μ', Σ, V))
    
    return xla, Matrix(Z'), B, Σ, Y
end

function simulate_multivariate_sparse(
    plinkname::String, n::Int, p::Int, k::Int, r::Int;
    seed::Int=2021,σg=0.6, σe=0.4,  βoverlap=2, 
    )
    # set seed
    Random.seed!(seed)
    
    # simulate `.bed` file with no missing data
    x = simulate_random_snparray("sim$seed/" * plinkname * ".bed", n, p)
    xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true)

    # intercept is the only nongenetic covariate
    z = ones(n, 1)
    intercepts = randn(r)' # each trait have different intercept

    # simulate response y, true model b, and the correct non-0 positions of b
    Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r, Zu=z*intercepts, overlap=2);
    
    return xla, Matrix(z'), true_b, true_Σ, Matrix(Y')
end

function make_bim_file(x::SnpLinAlg, name::String)
    p = size(x, 2)

    #create .bim file structure: https://www.cog-genomics.org/plink2/formats#bim
    open(name * ".bim", "w") do f
        for i in 1:p
            write(f, "1\tsnp$i\t0\t1\t1\t2\n")
        end
    end
end

function make_GEMMA_fam_file(x::SnpLinAlg, y::AbstractVecOrMat, name::String)
    ly = size(y, 1)
    n, p = size(x)

    # put 1st phenotypes in 6th column, 2nd phenotype in 7th column ... etc
    traits = size(y, 1)
    open(name * ".fam", "w") do f
        for i in 1:n
            write(f, "$i\t1\t0\t0\t1")
            for j in 1:traits
                write(f, "\t$(y[j, i])")
            end
            write(f, "\n")
        end
    end
end

function make_MVPLINK_fam_and_phen_file(x::SnpLinAlg, y::AbstractVecOrMat, name::String)
    ly = size(y, 1)
    n, p = size(x)

    # put a random phenotype in fam file
    traits = size(y, 1)
    open(name * ".fam", "w") do f
        for i in 1:n
            println(f, "$i\t1\t0\t0\t1\t1")
        end
    end

    # save phenotypes in separate `.phen` file
    open(name * ".phen", "w") do io
        print(io, "FID\tIID")
        for j in 1:traits
            print(io, "\tT$j")
        end
        print(io, "\n")
        for i in 1:n
            print(io, "$i\t1")
            for j in 1:traits
                write(io, "\t$(y[j, i])")
            end
            print(io, "\n")
        end
    end
end

"""
Computes power and false positive rates
- p: total number of SNPs
- correct_snps: Indices of the true causal SNPs
- detected_snps: Indices of SNPs that are significant after testing

returns: power, number of false positives, and false positive rate
"""
function power_and_fpr(p::Int, correct_snps::Vector{Int}, signif_snps::Vector{Int})
    power = length(signif_snps ∩ correct_snps) / length(correct_snps)
    FP = length(signif_snps) - length(signif_snps ∩ correct_snps) # number of false positives
    TN = p - length(signif_snps) # number of true negatives
    FPR = FP / (FP + TN)
    return power, FP, FPR
end

"""
- filename: gemma's output file name
- correct_snps: indices for real causal SNPs

returns: power, number of false positives, and false positive rate
"""
function process_gemma_result(filename, correct_snps)
    # read GEMMA result
    gemma_df = CSV.read(filename, DataFrame)
    snps = size(gemma_df, 1)

    # pvalues
    pval_wald = gemma_df[!, :p_wald]
#     pval_lrt = gemma_df[!, :p_lrt]
#     pval_score = gemma_df[!, :p_score]

    # estimated beta
    estim_β1 = gemma_df[!, :beta_1]
    estim_β2 = gemma_df[!, :beta_2]

    # estimated covariance matrix
    estim_σ11 = gemma_df[!, :Vbeta_1_1]
    estim_σ12 = gemma_df[!, :Vbeta_1_2]
    estim_σ22 = gemma_df[!, :Vbeta_2_2];

    # check how many real SNPs were recovered
    signif_snps = findall(x -> x ≤ 0.05 / snps, pval_wald) # gemma's selected snps
    @show signif_snps ∩ correct_snps

    # compute power, false positives, and false positive rate
    power_and_fpr(snps, correct_snps, signif_snps)
end

"""
- filename: mvPLINK's output file name
- correct_snps: indices for real causal SNPs

returns: power, number of false positives, and false positive rate
"""
function process_mvPLINK(filename, correct_snps)
    # read mvPLINK result
    mvplink_df = CSV.read(filename, DataFrame, delim=' ', ignorerepeated=true)
    snps = size(mvplink_df, 1)

    # get pvalues, possibly accounting for "NA"s
    if eltype(mvplink_df[!, :P]) == Float64
        pval = mvplink_df[!, :P]
    else
        mvplink_df[findall(x -> x == "NA", mvplink_df[!, :P]), :P] .= "1.0"
        pval = parse.(Float64, mvplink_df[!, :P])
    end

    # SNPs passing threshold
    signif_snps = findall(x -> x ≤ 0.05 / snps, pval)
    @show signif_snps ∩ correct_snps

    # compute power, false positives, and false positive rate
    power_and_fpr(snps, correct_snps, signif_snps)
end

"""
# Arguments
n = number of samples
p = number of SNPs
k = number of causal SNPs
r = number of traits

# Optional arguments
seed = random seed for reproducibility
σ2 = contribution of GRM
σe = random environmental effect
βoverlap = number of causal SNPs shared in all traits
"""
function one_simulation(
    n::Int, p::Int, k::Int, r::Int;
    seed::Int=2021, σg=0.6, σe=0.4, βoverlap=2
    )
    isdir("sim$seed") || mkdir("sim$seed")
    plinkname = "sim$(seed)_$(r)traits"

    # simulate data
    xla, Z, B, Σ, Y = simulate_multivariate_sparse(plinkname, n, p, k, r,
        seed=seed, σg=σg, σe=σe, βoverlap=βoverlap)
    correct_position = findall(!iszero, B)
    correct_snps = [x[1] for x in correct_position]
    make_bim_file(xla, "sim$(seed)/" * plinkname)

    # run IHT
    Random.seed!(seed)
    iht_time = @elapsed begin
        mses = cv_iht(Y, Transpose(xla), Z, parallel=true)
        iht_result = fit_iht(Y, Transpose(xla), Z, k=argmin(mses))
    end
    β1, β2 = iht_result.beta[1, :], iht_result.beta[2, :]
    detected_snps = findall(!iszero, β1) ∪ findall(!iszero, β2)
    iht_power, iht_FP, iht_FPR = power_and_fpr(size(B, 1), correct_snps, detected_snps)
    writedlm("sim$(seed)/iht_beta1.txt", β1)
    writedlm("sim$(seed)/iht_beta2.txt", β2)
    println("IHT power = $iht_power, FP = $iht_FP, FPR = $iht_FPR")

    # run MVPLINK
    phenofile = "sim$(seed)/" * plinkname * ".phen"
    outfile = "sim$(seed)/mvPLINK_sim$seed.mqfam.total"
    make_MVPLINK_fam_and_phen_file(xla, Y, "sim$(seed)/" * plinkname)
    mvplink_time = @elapsed run(`./plink.multivariate --bfile sim$(seed)/$plinkname --noweb --mult-pheno $phenofile --mqfam`)
    mv("plink.mqfam.total", outfile, force=true)
    mv("plink.log", "sim$(seed)/mvPLINK_sim$seed.log", force=true)
    mvPLINK_power, mvPLINK_FP, mvPLINK_FPR = process_mvPLINK(outfile, correct_snps)
    println("mvPLINK power = $mvPLINK_power, FP = $mvPLINK_FP, FPR = $mvPLINK_FPR")

    # run GEMMA
    make_GEMMA_fam_file(xla, Y, "sim$(seed)/" * plinkname)
    gemma_time = @elapsed begin
        run(`./gemma -bfile sim$(seed)/$plinkname -gk 1 -o $plinkname`)
        run(`./gemma -bfile sim$(seed)/$plinkname -k output/$(plinkname).cXX.txt -maf 0.0001 -lmm 1 -n 1 2 -o gemma.sim$seed`)
    end
    gemma_power, gemma_FP, gemma_FPR = process_gemma_result("output/gemma.sim$seed.assoc.txt", correct_snps)
    println("GEMMA power = $gemma_power, FP = $gemma_FP, FPR = $gemma_FPR")
    mv("output/gemma.sim$seed.assoc.txt", "sim$(seed)/gemma.sim$seed.assoc.txt")
    mv("output/gemma.sim$seed.log.txt", "sim$(seed)/gemma.sim$seed.log.txt")

    # save summary stats
    open("sim$(seed)/summary.txt", "w") do io
        println(io, "Simulation $seed summary")
        println(io, "n = $n, p = $p, k = $k, r = $r\n")
        println(io, "IHT time = $iht_time seconds, power = $iht_power, FP = $iht_FP, FPR = $iht_FPR")
        println(io, "mvPLINK time = $mvplink_time seconds, power = $mvPLINK_power, FP = $mvPLINK_FP, FPR = $mvPLINK_FPR")
        println(io, "GEMMA time = $gemma_time seconds, power = $gemma_power, FP = $gemma_FP, FPR = $gemma_FPR")
    end

    return nothing
end

n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs
r = 2     # number of traits

# seed = parse(Int, ARGS[1])
seed =2020
one_simulation(n, p, k, r, seed = seed)

┌ Info: Precompiling MendelIHT [921c7187-1484-5754-b919-5d3ed9ac03c4]
└ @ Base loading.jl:1278


2

In [13]:
function run_repeats(repeats::Int)
    for seed in 1:repeats
        # create .sh file to submit jobs
        open("run$seed.sh", "w") do io
            println(io, "#!/bin/bash")
            println(io, "#\$ -cwd")
            println(io, "# error = Merged with joblog")
            println(io, "#\$ -o joblog.\$JOB_ID")
            println(io, "#\$ -j y")
            println(io, "#\$ -pe shared 12")
            println(io, "#\$ -l h_rt=24:00:00,h_data=5G,exclusive")
            println(io, "# Email address to notify")
            println(io, "## \$ -M \$USER@mal")
            println(io, "# Notify when:")
            println(io, "#\$ -m bea")
            println(io, "")
            println(io, "#save job info on joblog:")
            println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
            println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
            println(io, "")
            println(io, "# load the job environment:")
            println(io, ". /u/local/Modules/default/init/modules.sh")
            println(io, "module load julia/1.5.4")
            println(io, "module li")
            println(io, "which julia")
            println(io, "")
            println(io, "# run code")
            println(io, "echo 'julia run.jl seed = $seed (run IHT/GEMMA/mvPLINK on simulated data (sparse model))'")
            println(io, "pwd; julia /u/home/b/biona001/multivariate/run.jl $seed")
            println(io, "")
            println(io, "#echo job info on joblog:")
            println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
            println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
            println(io, "#echo \" \"")
        end
        
        # submit job
        run(`qsub run$seed.sh`)
        rm("run$seed.sh", force=true)
        sleep(2)
    end
end
run_repeats(50)

make_run_sh (generic function with 1 method)

In [ ]:
"""
After performing n simulations using `run_repeats`, this function summarizes the result. 
"""
function summarize_repeats()
#     model = "polygenic"
    model = "sparse"
    
    iht_time, iht_power, iht_FP, iht_FPT = Float64[], Float64[], Float64[], Float64[]
    mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPT = Float64[], Float64[], Float64[], Float64[]
    gemma_time, gemma_power, gemma_FP, gemma_FPT = Float64[], Float64[], Float64[], Float64[]

    regex = r"= (\d+\.\d+) seconds, power = (\d+\.\d+), FP = (\d+), FPR = (0\.\d+)"
    for sim in 1:50
        if !isdir("$model/sim$sim") || !isfile("$model/sim$(sim)/summary.txt")
            println("Simulation $sim failed!")
            continue
        end
        open("$model/sim$(sim)/summary.txt", "r") do io
            readline(io); readline(io); readline(io)

            # parse IHT result
            iht = match(regex, readline(io))
            push!(iht_time, parse(Float64, iht[1]))
            push!(iht_power, parse(Float64, iht[2]))
            push!(iht_FP, parse(Float64, iht[3]))
            push!(iht_FPT, parse(Float64, iht[4]))

            # parse mvPLINK result
            mvPLINK = match(regex, readline(io))
            push!(mvPLINK_time, parse(Float64, mvPLINK[1]))
            push!(mvPLINK_power, parse(Float64, mvPLINK[2]))
            push!(mvPLINK_FP, parse(Float64, mvPLINK[3]))
            push!(mvPLINK_FPT, parse(Float64, mvPLINK[4]))

            # parse mvPLINK result
            gemma = match(regex, readline(io))
            push!(gemma_time, parse(Float64, gemma[1]))
            push!(gemma_power, parse(Float64, gemma[2]))
            push!(gemma_FP, parse(Float64, gemma[3]))
            push!(gemma_FPT, parse(Float64, gemma[4]))
        end
    end
    
    # save summary statistics
    open("$(model)_summary.txt", "w") do io
        println(io, "iht_time,iht_power,iht_FP,iht_FPT,mvPLINK_time,mvPLINK_power," * 
            "mvPLINK_FP,mvPLINK_FPT,gemma_time,gemma_power,gemma_FP,gemma_FPT")
        for i in eachindex(iht_time)
            print(io, iht_time[i], ',', iht_power[i], ',', iht_FP[i], ',', iht_FPT[i], ',')
            print(io, mvPLINK_time[i], ',', mvPLINK_power[i], ',', mvPLINK_FP[i], ',', mvPLINK_FPT[i], ',')
            print(io, gemma_time[i], ',', gemma_power[i], ',', gemma_FP[i], ',', gemma_FPT[i], "\n")
        end
    end
    
    return iht_time, iht_power, iht_FP, iht_FPT, 
        mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPT,
        gemma_time, gemma_power, gemma_FP, gemma_FPT
end
iht_time, iht_power, iht_FP, iht_FPT, 
    mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPT,
    gemma_time, gemma_power, gemma_FP, gemma_FPT = summarize_repeats()

In [16]:
# summary = CSV.read("sparse_summary.txt", DataFrame)
summary = CSV.read("polygenic_summary.txt", DataFrame)
iht_time, iht_power, iht_FP, iht_FPR = summary[!, 1], summary[!, 2], summary[!, 3], summary[!, 4]
mvPLINK_time, mvPLINK_power, mvPLINK_FP, mvPLINK_FPR = summary[!, 5], summary[!, 6], summary[!, 7], summary[!, 8]
gemma_time, gemma_power, gemma_FP, gemma_FPR = summary[!, 9], summary[!, 10], summary[!, 11], summary[!, 12];

## Polygenic summary

In [70]:
# time comparisons
@show mean(iht_time)
@show mean(mvPLINK_time)
@show mean(gemma_time)
[iht_time mvPLINK_time gemma_time]

mean(iht_time) = 102.12144187497917
mean(mvPLINK_time) = 9.455460785250002
mean(gemma_time) = 2872.545454084438


48×3 Array{Float64,2}:
 101.633    8.33465     31.3264
  73.6667   7.20927     19.9654
  79.2369   9.77717   9588.24
 155.951   11.1171      56.0372
 115.827    9.39161     38.0505
 143.396    9.82422     36.8143
 136.952    9.35495     41.7448
 101.223    9.79519   8627.74
  88.5298   9.40819     24.675
  81.8591   9.36133     26.0969
  79.4635   9.39698   9075.45
 112.114    9.35381   1215.72
  78.7716   9.34317     44.6783
   ⋮                 
 128.094    9.79237    867.71
  79.7074   9.78698   2963.97
 101.066    9.79456     40.6364
 124.03     9.36471     29.5194
 113.662    9.7644      44.715
  78.192    9.77141     42.9388
  83.149    9.36022   9080.8
 109.215    9.35909     41.7627
 120.249    9.85564  10124.6
  90.9305   9.36481   9069.79
  80.9163   9.35925   9082.79
  76.5582   9.79315    649.109

In [71]:
# power (proportion of true SNPs recovered) comparisons
@show mean(iht_power)
@show mean(mvPLINK_power)
@show mean(gemma_power)
[iht_power mvPLINK_power gemma_power]

mean(iht_power) = 0.7249999999999998
mean(mvPLINK_power) = 0.6270833333333333
mean(gemma_power) = 0.6250000000000001


48×3 Array{Float64,2}:
 0.7  0.5  0.5
 0.6  0.6  0.6
 0.8  0.6  0.6
 0.7  0.7  0.7
 0.6  0.6  0.6
 0.6  0.6  0.6
 0.7  0.6  0.6
 0.6  0.5  0.6
 0.8  0.7  0.7
 0.8  0.7  0.7
 0.8  0.8  0.8
 0.8  0.6  0.6
 0.8  0.6  0.6
 ⋮         
 0.5  0.5  0.5
 0.8  0.5  0.5
 0.7  0.6  0.6
 0.8  0.6  0.6
 0.6  0.4  0.4
 0.8  0.4  0.5
 0.7  0.6  0.6
 0.7  0.7  0.7
 0.8  0.6  0.6
 0.8  0.8  0.8
 0.8  0.6  0.6
 0.6  0.5  0.5

In [72]:
# Number of false positives
@show mean(iht_FP)
@show mean(mvPLINK_FP)
@show mean(gemma_FP)
[iht_FP mvPLINK_FP gemma_FP]

mean(iht_FP) = 2.125
mean(mvPLINK_FP) = 0.08333333333333333
mean(gemma_FP) = 0.0625


48×3 Array{Float64,2}:
  7.0  0.0  0.0
  3.0  0.0  0.0
  1.0  1.0  0.0
  0.0  1.0  1.0
  2.0  0.0  0.0
  0.0  1.0  1.0
  1.0  0.0  0.0
  5.0  0.0  0.0
  0.0  0.0  0.0
  1.0  0.0  0.0
  0.0  0.0  0.0
  5.0  0.0  0.0
  8.0  0.0  0.0
  ⋮         
  0.0  0.0  0.0
  4.0  0.0  0.0
 11.0  0.0  0.0
  0.0  0.0  0.0
  0.0  0.0  0.0
  8.0  0.0  0.0
  0.0  0.0  0.0
  0.0  0.0  0.0
  0.0  0.0  0.0
  2.0  0.0  0.0
  5.0  0.0  0.0
  0.0  0.0  0.0

In [73]:
# False positive rates
@show mean(iht_FPR)
@show mean(mvPLINK_FPR)
@show mean(gemma_FPR)
[iht_FPR mvPLINK_FPR gemma_FPR]

mean(iht_FPR) = 0.0002126586611114755
mean(mvPLINK_FPR) = 8.338753543997375e-6
mean(gemma_FPR) = 6.254169460213771e-6


48×3 Array{Float64,2}:
 0.00070049   0.0         0.0
 0.00030018   0.0         0.0
 0.00010008   0.00010006  0.0
 0.0          0.00010007  0.00010007
 0.00020012   0.0         0.0
 0.0          0.00010006  0.00010006
 0.00010007   0.0         0.0
 0.0005003    0.0         0.0
 0.0          0.0         0.0
 0.00010008   0.0         0.0
 0.0          0.0         0.0
 0.0005004    0.0         0.0
 0.000800641  0.0         0.0
 ⋮                        
 0.0          0.0         0.0
 0.00040032   0.0         0.0
 0.00110077   0.0         0.0
 0.0          0.0         0.0
 0.0          0.0         0.0
 0.000800641  0.0         0.0
 0.0          0.0         0.0
 0.0          0.0         0.0
 0.0          0.0         0.0
 0.00020016   0.0         0.0
 0.0005004    0.0         0.0
 0.0          0.0         0.0

## Sparse model

In [8]:
# time comparisons
@show mean(iht_time)
@show mean(mvPLINK_time)
@show mean(gemma_time)
[iht_time mvPLINK_time gemma_time]

mean(iht_time) = 169.51348402163046
mean(mvPLINK_time) = 12.378609219913043
mean(gemma_time) = 2504.4712677048697


46×3 Array{Float64,2}:
 163.511   13.9029      65.852
 202.617   12.5483      64.9008
  55.812    8.02798     29.9725
 169.562   14.2131      57.4539
  98.6313   7.1686      29.3038
 288.369   11.4195      35.4564
  96.624    7.94139     41.3202
 181.367   15.308       66.1481
 235.195   12.838       43.4029
 102.258    8.21246     57.614
  99.0519  10.1841   10240.3
 117.984   10.3483      32.0624
  87.4723  10.0278      27.5549
   ⋮                 
 210.073   10.0887      32.0299
 102.417   13.1494     344.458
 204.416   10.3118      33.9908
 258.506   14.6631      27.5109
 186.475   13.6468      62.6946
 276.6     11.4174    2146.12
 206.583   18.0525      58.7926
 135.894   15.6454     237.984
 303.609   17.9606     120.551
 132.759   14.7005   25465.1
 213.359   15.6241   20178.4
 102.632   14.7057     193.112

In [9]:
# power (proportion of true SNPs recovered) comparisons
@show mean(iht_power)
@show mean(mvPLINK_power)
@show mean(gemma_power)
[iht_power mvPLINK_power gemma_power]

mean(iht_power) = 0.7195652173913044
mean(mvPLINK_power) = 0.6239130434782609
mean(gemma_power) = 0.6239130434782609


46×3 Array{Float64,2}:
 0.7  0.5  0.5
 0.6  0.6  0.6
 0.8  0.6  0.6
 0.7  0.7  0.7
 0.6  0.6  0.6
 0.6  0.6  0.6
 0.8  0.6  0.6
 0.6  0.5  0.5
 0.8  0.7  0.7
 0.8  0.7  0.7
 0.8  0.8  0.8
 0.8  0.6  0.6
 0.8  0.6  0.6
 ⋮         
 0.5  0.5  0.5
 0.8  0.5  0.5
 0.7  0.6  0.6
 0.8  0.6  0.6
 0.5  0.4  0.4
 0.5  0.5  0.5
 0.8  0.4  0.5
 0.7  0.6  0.6
 0.7  0.7  0.7
 0.8  0.8  0.8
 0.8  0.6  0.6
 0.6  0.5  0.5

In [10]:
# Number of false positives
@show mean(iht_FP)
@show mean(mvPLINK_FP)
@show mean(gemma_FP)
[iht_FP mvPLINK_FP gemma_FP]

mean(iht_FP) = 1.7826086956521738
mean(mvPLINK_FP) = 0.08695652173913043
mean(gemma_FP) = 0.06521739130434782


46×3 Array{Float64,2}:
  4.0  0.0  0.0
  1.0  0.0  0.0
  1.0  1.0  0.0
  1.0  1.0  1.0
  3.0  0.0  0.0
  0.0  1.0  1.0
  0.0  0.0  0.0
  6.0  0.0  0.0
  3.0  0.0  0.0
  0.0  0.0  0.0
  0.0  0.0  0.0
  1.0  0.0  0.0
  3.0  0.0  0.0
  ⋮         
  0.0  0.0  0.0
  2.0  0.0  0.0
 10.0  0.0  0.0
  0.0  0.0  0.0
  0.0  0.0  0.0
  0.0  0.0  0.0
  7.0  0.0  0.0
  0.0  0.0  0.0
  0.0  0.0  0.0
  2.0  0.0  0.0
  6.0  0.0  0.0
  0.0  0.0  0.0

In [11]:
# False positive rates
@show mean(iht_FPR)
@show mean(mvPLINK_FPR)
@show mean(gemma_FPR)
[iht_FPR mvPLINK_FPR gemma_FPR]

mean(iht_FPR) = 0.00017839531960002123
mean(mvPLINK_FPR) = 8.701308045910303e-6
mean(gemma_FPR) = 6.526089871527413e-6


46×3 Array{Float64,2}:
 0.00040028  0.0         0.0
 0.00010006  0.0         0.0
 0.00010008  0.00010006  0.0
 0.00010007  0.00010007  0.00010007
 0.00030018  0.0         0.0
 0.0         0.00010006  0.00010006
 0.0         0.0         0.0
 0.00060036  0.0         0.0
 0.00030024  0.0         0.0
 0.0         0.0         0.0
 0.0         0.0         0.0
 0.00010008  0.0         0.0
 0.00030024  0.0         0.0
 ⋮                       
 0.0         0.0         0.0
 0.00020016  0.0         0.0
 0.0010007   0.0         0.0
 0.0         0.0         0.0
 0.0         0.0         0.0
 0.0         0.0         0.0
 0.00070056  0.0         0.0
 0.0         0.0         0.0
 0.0         0.0         0.0
 0.00020016  0.0         0.0
 0.00060048  0.0         0.0
 0.0         0.0         0.0